# Crypto Forecasting 

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
import numpy as np
import hvplot.pandas
%matplotlib inline

# Stocks Data

In [2]:
# Load .env environment variables
load_dotenv("alpaca.env")

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [4]:
# Set start and end dates of five years back from today.
start_date = pd.Timestamp("2018-05-01", tz = "America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz = "America/New_York").isoformat()

# Set the tickers
tickers = ["AAPL", "MSFT", "AMD"]
timeframe = "1D"

# Get current closing prices 
stocks_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
AAPL = stocks_df[stocks_df["symbol"]=="AAPL"].drop("symbol", axis=1)
MSFT = stocks_df[stocks_df["symbol"]=="MSFT"].drop("symbol", axis=1)
AMD = stocks_df[stocks_df["symbol"]=="AMD"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
stocks_df = pd.concat([AAPL, MSFT, AMD],axis=1, keys=["AAPL","MSFT", "AMD"])

# Preview DataFrame
stocks_df

AAPL                                       \
                               open     high       low   close    volume   
timestamp                                                                  
2018-05-01 04:00:00+00:00  166.4102  169.200  165.2700  169.10  53569474   
2018-05-02 04:00:00+00:00  175.2250  177.750  173.8000  176.57  66851451   
2018-05-03 04:00:00+00:00  175.8800  177.500  174.4411  176.89  34068446   
2018-05-04 04:00:00+00:00  178.2500  184.250  178.1700  183.83  56204195   
2018-05-07 04:00:00+00:00  185.1800  187.670  184.7500  185.16  42613114   
...                             ...      ...       ...     ...       ...   
2023-04-25 04:00:00+00:00  165.1900  166.305  163.7300  163.77  48849358   
2023-04-26 04:00:00+00:00  163.9000  165.280  162.8000  163.76  45524694   
2023-04-27 04:00:00+00:00  165.1900  168.560  165.1900  168.41  64891617   
2023-04-28 04:00:00+00:00  168.4900  169.850  167.8801  169.68  53888026   
2023-05-01 04:00:00+00:00  169.2800  170.450  168.6400  169.59  52546063   

                                                     MSFT                   \
                          trade_count        vwap    open     high     low   
timestamp                                                                    
2018-05-01 04:00:00+00:00      353863  169.130276   93.21   95.290   92.79   
2018-05-02 04:00:00+00:00      437602  176.126868   94.99   95.170   93.19   
2018-05-03 04:00:00+00:00      246062  176.100682   92.96   94.925   92.45   
2018-05-04 04:00:00+00:00      370771  182.554203   93.32   95.370   92.92   
2018-05-07 04:00:00+00:00      267449  186.216900   95.17   96.710   95.10   
...                               ...         ...     ...      ...     ...   
2023-04-25 04:00:00+00:00      502773  164.647271  279.51  281.600  275.37   
2023-04-26 04:00:00+00:00      494174  163.908156  296.70  299.570  292.73   
2023-04-27 04:00:00+00:00      575890  167.319460  295.97  305.200  295.25   
2023-04-28 04:00:00+00:00      513328  168.948227  304.01  308.930  303.31   
2023-05-01 04:00:00+00:00      541577  169.657746  306.97  308.600  305.15   

                           ...                                       AMD  \
                           ...    volume trade_count        vwap    open   
timestamp                  ...                                             
2018-05-01 04:00:00+00:00  ...  31409017      209726   94.199608  10.830   
2018-05-02 04:00:00+00:00  ...  27471134      188110   94.016453  11.100   
2018-05-03 04:00:00+00:00  ...  31142548      238268   93.702789  10.902   
2018-05-04 04:00:00+00:00  ...  22531383      165938   94.745049  10.880   
2018-05-07 04:00:00+00:00  ...  24262669      172194   96.187561  11.320   
...                        ...       ...         ...         ...     ...   
2023-04-25 04:00:00+00:00  ...  47004633      627981  280.958848  86.900   
2023-04-26 04:00:00+00:00  ...  64681706      869828  296.027735  86.510   
2023-04-27 04:00:00+00:00  ...  46463058      650724  302.549963  86.410   
2023-04-28 04:00:00+00:00  ...  31737282      485116  306.123532  87.020   
2023-05-01 04:00:00+00:00  ...  21329194      317112  306.613708  91.030   

                                                                         \
                             high      low  close    volume trade_count   
timestamp                                                                 
2018-05-01 04:00:00+00:00  11.150  10.7700  11.13  43441602       90980   
2018-05-02 04:00:00+00:00  11.155  10.9700  10.97  29267466       77517   
2018-05-03 04:00:00+00:00  11.060  10.8300  10.93  33310596       89301   
2018-05-04 04:00:00+00:00  11.290  10.8701  11.28  47885590      120350   
2018-05-07 04:00:00+00:00  11.680  11.3000  11.59  54003947      132612   
...                           ...      ...    ...       ...         ...   
2023-04-25 04:00:00+00:00  87.060  83.7600  83.80  52893533      342477   
2023-04-26 04:00:00+00:00  87.350  85.3200  85.94  4781519

In [5]:
# Count null
stocks_df.isnull().sum()

AAPL  open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
MSFT  open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
AMD   open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
dtype: int64

# Stocks Monte Carlo Simulation

In [ ]:
# Set number of simulations
num_sims = 100

# Configure a Monte Carlo simulation to forecast 5 years daily returns
MC_stocks = MCSimulation(
    portfolio_data = stocks_df,
    weights = [1/3,1/3,1/3],
    num_simulation = num_sims,
    num_trading_days = 252*5
)

In [ ]:
# Run Monte Carlo simulations to forecast 5 years daily returns
MC_stocks.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
stocks_line_plot = MC_stocks.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
stocks_dist_plot = MC_stocks.plot_distribution()

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
stocks_simulation_data_df = MC_stocks.simulated_return

# Convert the DataFrame to long-form
stocks_long_form_data = stocks_simulation_data_df.stack().reset_index()
stocks_long_form_data.columns = ['Time', 'Simulation', 'Value']

# Create an interactive line plot using Holoviews
stocks_curves = {sim: hv.Curve(stocks_long_form_data[stocks_long_form_data['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in stocks_long_form_data['Simulation'].unique()}
stocks_interactive_plot = hv.NdOverlay(stocks_curves, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
stocks_interactive_plot.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)

In [ ]:
# Compute summary statistics from the simulated daily returns
stocks_returns_data = {
    'mean': list(MC_stocks.simulated_return.mean(axis=1)),
    'median': list(MC_stocks.simulated_return.median(axis=1)),
    'min': list(MC_stocks.simulated_return.min(axis=1)),
    'max': list(MC_stocks.simulated_return.max(axis=1))
}

# Create a DataFrame with the summary statistics
stocks_simulated_returns = pd.DataFrame(stocks_returns_data)

# Display sample data
stocks_simulated_returns.head()

In [ ]:
stocks_simulated_returns[['mean','median']].plot(title='Simulated cumulative return behavior of AAPL, MSFT, AMD over five years')

In [ ]:
stocks_tbl = MC_stocks.summarize_cumulative_return()

print(stocks_tbl)

In [ ]:
stocks_initial_investment = 10000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments in AAPL, MSFT, AMD stocks
ci_lower = round(stocks_tbl[8]*10000,2)
ci_upper = round(stocks_tbl[9]*10000,2)

# Print results
print(f"There is a 95% chance that an initial investment of {stocks_initial_investment} in the portfolio over the next year will end within in the range of ${ci_lower} and ${ci_upper}.")

In [ ]:
# Extract the closing prices of the three stocks
aapl_prices = stocks_df["AAPL"]["close"]
msft_prices = stocks_df["MSFT"]["close"]
amd_prices = stocks_df["AMD"]["close"]

# Calculate the daily returns of each stock
aapl_returns = np.log(aapl_prices) - np.log(aapl_prices.shift(1))
msft_returns = np.log(msft_prices) - np.log(msft_prices.shift(1))
amd_returns = np.log(amd_prices) - np.log(amd_prices.shift(1))

In [ ]:
# Calculate the weighted daily returns
stocks_weighted_returns = (1/3) * (aapl_returns + msft_returns + amd_returns)
stocks_weighted_returns.dropna()

# Convert the daily returns into cumulative returns
stocks_cumulative_returns = (1 + stocks_weighted_returns).cumprod()
stocks_cumulative_returns.dropna()

In [ ]:
# Calculate the portfolio value over time
stocks_portfolio_value = 10000 * stocks_cumulative_returns

# Retrieve the portfolio value at the last date
stocks_final_value = stocks_portfolio_value[1258]
print(f"If you had invested 10,000 evenly into this portfolio on May 1st, 2018 it would be worth {stocks_final_value} today.")

In [ ]:
stocks_portfolio_value = stocks_portfolio_value.to_frame()

In [ ]:
stocks_portfolio_value.hvplot(title = "Cummulative Returns of Stocks Portfolio", xlabel = "Date", ylabel = "Portfolio Value", width = 800, height = 400)

# Forex Data

In [6]:
from pathlib import Path

In [7]:
# Reading EUR
EUR_path = Path("../resources/EURUSD.csv")
EUR_df = pd.read_csv(EUR_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
EUR_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-27,1.36015,1.36820,1.35850,1.36235,1023126
2007-04-28,1.36235,1.36540,1.36155,1.36520,95014
2007-04-30,1.36305,1.36777,1.35884,1.36581,545597
2007-05-01,1.36586,1.36711,1.35897,1.35984,664031
2007-05-02,1.35986,1.36210,1.35598,1.35962,659075


In [8]:
# Reading GBP
GBP_path = Path("../resources/GBPUSD.csv")
GBP_df = pd.read_csv(GBP_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
GBP_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-26,1.99140,1.99180,1.9897,1.9901,18697
2007-04-27,1.99010,2.00430,1.9864,1.9984,492786
2007-04-28,1.99840,1.99950,1.9954,1.9976,90222
2007-04-30,1.99578,2.00310,1.9890,1.9998,426199
2007-05-01,1.99980,2.00745,1.9980,1.9991,601529


In [9]:
# Reading JPY
JPY_path = Path("../resources/JPYUSD.csv")
JPY_df = pd.read_csv(JPY_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
JPY_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-06,118.830,119.390,118.725,119.340,157979
2007-04-07,119.340,119.360,119.210,119.245,10085
2007-04-09,119.245,119.380,119.175,119.280,212315
2007-04-10,119.280,119.385,118.760,119.065,700242
2007-04-11,119.070,119.520,118.870,119.360,753655


In [10]:
# Concatenate the DataFrames
forex_df = pd.concat([EUR_df, GBP_df, JPY_df],axis=1, keys=["EUR","GBP", "JPY"])

# Preview DataFrame
forex_df

EUR                                           GBP           \
               Open     High      Low    Close    Volume     Open     High   
Date                                                                         
2007-04-06      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-07      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-09      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-10      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-11      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
...             ...      ...      ...      ...       ...      ...      ...   
2023-04-29  1.10394  1.10406  1.10119  1.10144   48265.0  1.25801  1.25837   
2023-05-01  1.10240  1.10355  1.09661  1.09692  209835.0  1.25538  1.25689   
2023-05-02  1.09692  1.10066  1.09422  1.09965  354234.0  1.24929  1.25120   
2023-05-03  1.09964  1.10593  1.09940  1.10478  281057.0  1.24730  1.25531   
2023-05-04  1.10478  1.10916  1.10371  1.10679  166874.0  1.25446  1.25917   

                                            JPY                             \
                Low    Close    Volume     Open     High      Low    Close   
Date                                                                         
2007-04-06      NaN      NaN       NaN  118.830  119.390  118.725  119.340   
2007-04-07      NaN      NaN       NaN  119.340  119.360  119.210  119.245   
2007-04-09      NaN      NaN       NaN  119.245  119.380  119.175  119.280   
2007-04-10      NaN      NaN       NaN  119.280  119.385  118.760  119.065   
2007-04-11      NaN      NaN       NaN  119.070  119.520  118.870  119.360   
...             ...      ...       ...      ...      ...      ...      ...   
2023-04-29  1.25572  1.25608   20129.0  135.995  136.348  135.960  136.243   
2023-05-01  1.24863  1.24929  129575.0  136.105  137.409  136.105  137.354   
2023-05-02  1.24352  1.24734  144194.0  137.354  137.772  136.387  136.464   
2023-05-03  1.24629  1.25446  123853.0  136.464  136.700  135.067  135.312   
2023-05-04  1.25374  1.25840  135690.0  135.313  135.704  134.345  134.537   

                      
              Volume  
Date                  
2007-04-06  157979.0  
2007-04-07   10085.0  
2007-04-09  212315.0  
2007-04-10  700242.0  
2007-04-11  753655.0  
...              ...  
2023-04-29   70947.0  
2023-05-01  430547.0  
2023-05-02  607095.0  
2023-05-03  551343.0  
2023-05-04  307813.0  

[5029 rows x 15 columns]

In [11]:
# Choose the desired timeframe
forex_df = forex_df.loc["2018-05-01":"2023-05-01"]
forex_df

EUR                                           GBP           \
               Open     High      Low    Close    Volume     Open     High   
Date                                                                         
2018-05-01  1.20735  1.21062  1.19814  1.19995  208172.0  1.37496  1.37765   
2018-05-02  1.19994  1.20316  1.19531  1.19672  245713.0  1.36091  1.36656   
2018-05-03  1.19674  1.20255  1.19378  1.19706  280127.0  1.36084  1.36519   
2018-05-04  1.19704  1.20041  1.19107  1.19452  235191.0  1.35681  1.35856   
2018-05-05  1.19451  1.19659  1.19365  1.19550   42242.0  1.35337  1.35522   
...             ...      ...      ...      ...       ...      ...      ...   
2023-04-26  1.09685  1.10952  1.09641  1.10509  395863.0  1.23951  1.25150   
2023-04-27  1.10509  1.10634  1.09920  1.10154  358245.0  1.24837  1.24992   
2023-04-28  1.10156  1.10449  1.09624  1.10394  385123.0  1.24692  1.25804   
2023-04-29  1.10394  1.10406  1.10119  1.10144   48265.0  1.25801  1.25837   
2023-05-01  1.10240  1.10355  1.09661  1.09692  209835.0  1.25538  1.25689   

                                            JPY                             \
                Low    Close    Volume     Open     High      Low    Close   
Date                                                                         
2018-05-01  1.35880  1.36092  118028.0  109.287  109.794  109.086  109.686   
2018-05-02  1.35804  1.36080  144116.0  109.686  110.036  109.515  109.903   
2018-05-03  1.35377  1.35684  185202.0  109.902  109.963  108.929  109.048   
2018-05-04  1.34861  1.35338  139486.0  109.049  109.272  108.648  109.166   
2018-05-05  1.35232  1.35253   25412.0  109.167  109.202  108.957  109.101   
...             ...      ...       ...      ...      ...      ...      ...   
2023-04-26  1.23908  1.24836  145123.0  133.967  134.014  133.013  133.722   
2023-04-27  1.24359  1.24688  158894.0  133.723  134.197  133.204  134.054   
2023-04-28  1.24462  1.25803  184504.0  134.052  136.560  133.335  135.994   
2023-04-29  1.25572  1.25608   20129.0  135.995  136.348  135.960  136.243   
2023-05-01  1.24863  1.24929  129575.0  136.105  137.409  136.105  137.354   

                      
              Volume  
Date                  
2018-05-01  102630.0  
2018-05-02  107271.0  
2018-05-03  131190.0  
2018-05-04  137849.0  
2018-05-05   18969.0  
...              ...  
2023-04-26  714287.0  
2023-04-27  587795.0  
2023-04-28  601996.0  
2023-04-29   70947.0  
2023-05-01  430547.0  

[1564 rows x 15 columns]

In [12]:
# Count null
forex_df.isnull().sum()

EUR  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
GBP  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
JPY  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
dtype: int64

# Crypto

In [13]:
# Reading BTC
BTC_path = Path("../resources/BTC-USD.csv")
BTC_df = pd.read_csv(BTC_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
BTC_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-05-01,9251.469727,9255.879883,8891.049805,9119.009766,9119.009766,7.713020e+09
2018-05-02,9104.599609,9256.519531,9015.139648,9235.919922,9235.919922,7.558160e+09
2018-05-03,9233.969727,9798.330078,9188.150391,9743.860352,9743.860352,1.020730e+10
2018-05-04,9695.500000,9779.200195,9585.959961,9700.759766,9700.759766,8.217830e+09
2018-05-05,9700.280273,9964.500000,9695.120117,9858.150391,9858.150391,7.651940e+09


In [14]:
# Reading ETH
ETH_path = Path("../resources/ETH-USD.csv")
ETH_df = pd.read_csv(ETH_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
ETH_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-05-04,776.775024,803.745972,762.632019,785.624023,785.624023,3533410048
2018-05-05,784.583008,827.455017,784.237000,816.119995,816.119995,3035040000
2018-05-06,816.088013,835.057007,764.882996,792.310974,792.310974,3105570048
2018-05-07,793.338989,795.757996,710.177979,753.724976,753.724976,4316120064
2018-05-08,755.008972,774.249023,728.129028,752.856995,752.856995,2920489984


In [15]:
# Reading DOGE
DOGE_path = Path("../resources/DOGE-USD.csv")
DOGE_df = pd.read_csv(DOGE_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
DOGE_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-05-04,0.005533,0.005555,0.005205,0.005269,0.005269,14434200
2018-05-05,0.005267,0.005347,0.005026,0.005180,0.005180,30335800
2018-05-06,0.005202,0.005259,0.004725,0.004970,0.004970,33446900
2018-05-07,0.004966,0.005017,0.004632,0.004817,0.004817,25891500
2018-05-08,0.004833,0.005356,0.004792,0.005303,0.005303,40660100


In [16]:
# Concatenate the DataFrames
crypto_df = pd.concat([BTC_df, ETH_df, DOGE_df],axis=1, keys=["BTC","ETH", "DOGE"])

# Preview DataFrame
crypto_df

BTC                                            \
                    Open          High           Low         Close   
Date                                                                 
2018-05-01   9251.469727   9255.879883   8891.049805   9119.009766   
2018-05-02   9104.599609   9256.519531   9015.139648   9235.919922   
2018-05-03   9233.969727   9798.330078   9188.150391   9743.860352   
2018-05-04   9695.500000   9779.200195   9585.959961   9700.759766   
2018-05-05   9700.280273   9964.500000   9695.120117   9858.150391   
...                  ...           ...           ...           ...   
2023-04-30  29245.515625  29952.029297  29114.021484  29268.806641   
2023-05-01           NaN           NaN           NaN           NaN   
2023-05-02  28011.582031  28141.708984  27990.740234  28047.042969   
2023-05-03           NaN           NaN           NaN           NaN   
2023-05-04           NaN           NaN           NaN           NaN   

                                                ETH                            \
               Adj Close        Volume         Open         High          Low   
Date                                                                            
2018-05-01   9119.009766  7.713020e+09          NaN          NaN          NaN   
2018-05-02   9235.919922  7.558160e+09          NaN          NaN          NaN   
2018-05-03   9743.860352  1.020730e+10          NaN          NaN          NaN   
2018-05-04   9700.759766  8.217830e+09   776.775024   803.745972   762.632019   
2018-05-05   9858.150391  7.651940e+09   784.583008   827.455017   784.237000   
...                  ...           ...          ...          ...          ...   
2023-04-30  29268.806641  1.465220e+10  1908.741333  1938.417969  1876.924316   
2023-05-01           NaN           NaN  1868.891113  1886.206421  1809.193237   
2023-05-02  28047.042969  1.829745e+10  1831.726563  1879.759521  1824.327026   
2023-05-03           NaN           NaN  1870.687134  1915.504150  1845.871826   
2023-05-04           NaN           NaN  1905.153809  1915.088379  1870.087524   

                                                        DOGE            \
                  Close    Adj Close        Volume      Open      High   
Date                                                                     
2018-05-01          NaN          NaN           NaN       NaN       NaN   
2018-05-02          NaN          NaN           NaN       NaN       NaN   
2018-05-03          NaN          NaN           NaN       NaN       NaN   
2018-05-04   785.624023   785.624023  3.533410e+09  0.005533  0.005555   
2018-05-05   816.119995   816.119995  3.035040e+09  0.005267  0.005347   
...                 ...          ...           ...       ...       ...   
2023-04-30  1876.924316  1876.924316  6.539642e+09  0.081627  0.081701   
2023-05-01  1831.954834  1831.954834  8.625783e+09  0.079760  0.079996   
2023-05-02  1870.789307  1870.789307  7.536786e+09  0.078822  0.079045   
2023-05-03  1904.651855  1904.651855  8.306391e+09  0.078806  0.080527   
2023-05-04  1876.341919  1876.341919  7.064044e+09  0.079729  0.079729   

                                                       
                 Low     Close Adj Close       Volume  
Date                                                   
2018-05-01       NaN       NaN       NaN          NaN  
2018-05-02       NaN       NaN       NaN          NaN  
2018-05-03       NaN       NaN       NaN          NaN  
2018-05-04  0.005205  0.005269  0.005269   14434200.0  
2018-05-05  0.005026  0.005180  0.005180   30335800.0  
...              ...       ...       ...          ...  
2023-04-30  0.079628  0.079769  0.079769  264245527.0  
2023-05-01  0.077628  0.078820  0.078820  325694977.0  
2023-05-02  0.077832  0.078806  0.078806  234651604.0  
2023-05-03  0.077328  0.079713  0.079713  306950726.0  
2023-05-04  0.078156  0.078340  0.078340  283550080.0  

[1830 rows x 18 columns]

In [17]:
# Choose the desired timeframe
crypto_df = crypto_df.loc["2018-05-01":"2023-05-01"]
crypto_df

BTC                                            \
                    Open          High           Low         Close   
Date                                                                 
2018-05-01   9251.469727   9255.879883   8891.049805   9119.009766   
2018-05-02   9104.599609   9256.519531   9015.139648   9235.919922   
2018-05-03   9233.969727   9798.330078   9188.150391   9743.860352   
2018-05-04   9695.500000   9779.200195   9585.959961   9700.759766   
2018-05-05   9700.280273   9964.500000   9695.120117   9858.150391   
...                  ...           ...           ...           ...   
2023-04-27  28428.464844  29871.546875  28402.886719  29473.787109   
2023-04-28  29481.013672  29572.791016  28929.609375  29340.261719   
2023-04-29  29336.566406  29452.455078  29088.042969  29248.488281   
2023-04-30  29245.515625  29952.029297  29114.021484  29268.806641   
2023-05-01           NaN           NaN           NaN           NaN   

                                                ETH                            \
               Adj Close        Volume         Open         High          Low   
Date                                                                            
2018-05-01   9119.009766  7.713020e+09          NaN          NaN          NaN   
2018-05-02   9235.919922  7.558160e+09          NaN          NaN          NaN   
2018-05-03   9743.860352  1.020730e+10          NaN          NaN          NaN   
2018-05-04   9700.759766  8.217830e+09   776.775024   803.745972   762.632019   
2018-05-05   9858.150391  7.651940e+09   784.583008   827.455017   784.237000   
...                  ...           ...          ...          ...          ...   
2023-04-27  29473.787109  2.715345e+10  1866.693237  1936.945190  1863.147949   
2023-04-28  29340.261719  1.754446e+10  1909.273071  1923.199219  1876.291870   
2023-04-29  29248.488281  1.066263e+10  1892.565063  1916.973755  1887.990479   
2023-04-30  29268.806641  1.465220e+10  1908.741333  1938.417969  1876.924316   
2023-05-01           NaN           NaN  1868.891113  1886.206421  1809.193237   

                                                        DOGE            \
                  Close    Adj Close        Volume      Open      High   
Date                                                                     
2018-05-01          NaN          NaN           NaN       NaN       NaN   
2018-05-02          NaN          NaN           NaN       NaN       NaN   
2018-05-03          NaN          NaN           NaN       NaN       NaN   
2018-05-04   785.624023   785.624023  3.533410e+09  0.005533  0.005555   
2018-05-05   816.119995   816.119995  3.035040e+09  0.005267  0.005347   
...                 ...          ...           ...       ...       ...   
2023-04-27  1908.786377  1908.786377  1.400815e+10  0.078638  0.080950   
2023-04-28  1892.512817  1892.512817  7.691760e+09  0.080193  0.080576   
2023-04-29  1908.916992  1908.916992  4.796651e+09  0.080366  0.081846   
2023-04-30  1876.924316  1876.924316  6.539642e+09  0.081627  0.081701   
2023-05-01  1831.954834  1831.954834  8.625783e+09  0.079760  0.079996   

                                                       
                 Low     Close Adj Close       Volume  
Date                                                   
2018-05-01       NaN       NaN       NaN          NaN  
2018-05-02       NaN       NaN       NaN          NaN  
2018-05-03       NaN       NaN       NaN          NaN  
2018-05-04  0.005205  0.005269  0.005269   14434200.0  
2018-05-05  0.005026  0.005180  0.005180   30335800.0  
...              ...       ...       ...          ...  
2023-04-27  0.078527  0.080190  0.080190  435944109.0  
2023-04-28  0.079423  0.080364  0.080364  258987983.0  
2023-04-29  0.080303  0.081630  0.081630  268639541.0  
2023-04-30  0.079628  0.079769  0.079769  264245527.0  
2023-05-01  0.077628  0.078820  0.078820  325694977.0  

[1827 rows x 18 columns]

In [18]:
crypto_df = crypto_df.dropna()
crypto_df

BTC                                            \
                    Open          High           Low         Close   
Date                                                                 
2018-05-04   9695.500000   9779.200195   9585.959961   9700.759766   
2018-05-05   9700.280273   9964.500000   9695.120117   9858.150391   
2018-05-06   9845.309570   9940.139648   9465.250000   9654.799805   
2018-05-07   9645.669922   9665.849609   9231.530273   9373.009766   
2018-05-08   9380.870117   9462.750000   9127.769531   9234.820313   
...                  ...           ...           ...           ...   
2023-04-26  28300.058594  29995.837891  27324.548828  28422.701172   
2023-04-27  28428.464844  29871.546875  28402.886719  29473.787109   
2023-04-28  29481.013672  29572.791016  28929.609375  29340.261719   
2023-04-29  29336.566406  29452.455078  29088.042969  29248.488281   
2023-04-30  29245.515625  29952.029297  29114.021484  29268.806641   

                                                ETH                            \
               Adj Close        Volume         Open         High          Low   
Date                                                                            
2018-05-04   9700.759766  8.217830e+09   776.775024   803.745972   762.632019   
2018-05-05   9858.150391  7.651940e+09   784.583008   827.455017   784.237000   
2018-05-06   9654.799805  7.222280e+09   816.088013   835.057007   764.882996   
2018-05-07   9373.009766  7.394020e+09   793.338989   795.757996   710.177979   
2018-05-08   9234.820313  7.415870e+09   755.008972   774.249023   728.129028   
...                  ...           ...          ...          ...          ...   
2023-04-26  28422.701172  3.185424e+10  1866.148926  1962.567627  1792.463867   
2023-04-27  29473.787109  2.715345e+10  1866.693237  1936.945190  1863.147949   
2023-04-28  29340.261719  1.754446e+10  1909.273071  1923.199219  1876.291870   
2023-04-29  29248.488281  1.066263e+10  1892.565063  1916.973755  1887.990479   
2023-04-30  29268.806641  1.465220e+10  1908.741333  1938.417969  1876.924316   

                                                        DOGE            \
                  Close    Adj Close        Volume      Open      High   
Date                                                                     
2018-05-04   785.624023   785.624023  3.533410e+09  0.005533  0.005555   
2018-05-05   816.119995   816.119995  3.035040e+09  0.005267  0.005347   
2018-05-06   792.310974   792.310974  3.105570e+09  0.005202  0.005259   
2018-05-07   753.724976   753.724976  4.316120e+09  0.004966  0.005017   
2018-05-08   752.856995   752.856995  2.920490e+09  0.004833  0.005356   
...                 ...          ...           ...       ...       ...   
2023-04-26  1866.564209  1866.564209  1.435652e+10  0.079848  0.083372   
2023-04-27  1908.786377  1908.786377  1.400815e+10  0.078638  0.080950   
2023-04-28  1892.512817  1892.512817  7.691760e+09  0.080193  0.080576   
2023-04-29  1908.916992  1908.916992  4.796651e+09  0.080366  0.081846   
2023-04-30  1876.924316  1876.924316  6.539642e+09  0.081627  0.081701   

                                                       
                 Low     Close Adj Close       Volume  
Date                                                   
2018-05-04  0.005205  0.005269  0.005269   14434200.0  
2018-05-05  0.005026  0.005180  0.005180   30335800.0  
2018-05-06  0.004725  0.004970  0.004970   33446900.0  
2018-05-07  0.004632  0.004817  0.004817   25891500.0  
2018-05-08  0.004792  0.005303  0.005303   40660100.0  
...              ...       ...       ...          ...  
2023-04-26  0.076151  0.078623  0.078623  722718207.0  
2023-04-27  0.078527  0.080190  0.080190  435944109.0  
2023-04-28  0.079423  0.080364  0.080364  258987983.0  
2023-04-29  0.080303  0.081630  0.081630  268639541.0  
2023-04-30  0.079628  0.079769  0.079769  264245527.0  

[1823 rows x 18 columns]

In [ ]:
# Reading BTC
BTC_path = Path("../resources/BTC-USD.csv")
BTC_df = pd.read_csv(BTC_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
BTC_df.head()

In [ ]:
# Reading ETH
ETH_path = Path("../resources/ETH-USD.csv")
ETH_df = pd.read_csv(ETH_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
ETH_df.head()

In [ ]:
# Reading DOGE
DOGE_path = Path("../resources/DOGE-USD.csv")
DOGE_df = pd.read_csv(DOGE_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
DOGE_df.head()

In [ ]:
# Concatenate the DataFrames
crypto_df = pd.concat([BTC_df, ETH_df, DOGE_df],axis=1, keys=["BTC","ETH", "DOGE"])

# Preview DataFrame
crypto_df

In [ ]:
# Choose the desired timeframe
crypto_df = crypto_df.loc["2018-05-01":"2023-05-01"]
crypto_df

In [ ]:
crypto_df = crypto_df.dropna()
crypto_df

# Commodities

In [ ]:
gold_df = pd.read_csv('../resources/gold.csv')
copper_df = pd.read_csv('../resources/copper.csv')
silver_df = pd.read_csv('../resources/silver.csv')

In [ ]:
gold_df.tail()

In [ ]:
gold_df.rename(columns={'Date': 'timestamp'}, inplace=True)
gold_df.rename(columns={'Close/Last': 'close'}, inplace=True)
silver_df.rename(columns={'Date': 'timestamp'}, inplace=True)
silver_df.rename(columns={'Close/Last': 'close'}, inplace=True)
copper_df.rename(columns={'Date': 'timestamp'}, inplace=True)
copper_df.rename(columns={'Close/Last': 'close'}, inplace=True)

In [ ]:
gold_df.set_index('timestamp', inplace=True)
silver_df.set_index('timestamp', inplace=True)
copper_df.set_index('timestamp', inplace=True)

In [ ]:
gold_df.head()

In [ ]:
copper_df.head()

In [ ]:
silver_df.head()

In [ ]:
commodities_df = pd.concat([gold_df,silver_df,copper_df],axis=1, keys =["GOLD","SILVER","COPPER"])
commodities_df.head()

In [ ]:
?MCSimulation

In [ ]:
MC_5_year = MCSimulation(
    portfolio_data =commodities_df ,
    weights =[1/3,1/3,1/3],
    num_simulation = 100,
    num_trading_days = 252*5
)

In [ ]:
MC_5_year.portfolio_data.head()

In [ ]:
MC_5_year.calc_cumulative_return()

In [ ]:
line_plot = MC_5_year.plot_simulation()
line_plot.get_figure().savefig("MC_5_year_sim_plot.png",bbox_inches='tight')

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


In [ ]:
simulation_data_df = MC_5_year.simulated_return

# Convert the DataFrame to long-form
long_form_data = simulation_data_df.stack().reset_index()
long_form_data.columns = ['Time', 'Simulation', 'Value']

# Import Holoviews and enable the Bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Create an interactive line plot using Holoviews
curves = {sim: hv.Curve(long_form_data[long_form_data['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in long_form_data['Simulation'].unique()}
interactive_plot = hv.NdOverlay(curves, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
interactive_plot.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)



In [ ]:
tbl_commodities =  MC_5_year.summarize_cumulative_return()

In [ ]:
tbl_commodities

In [ ]:
MC_5_year.calc_cumulative_return()

In [ ]:
commodity_simulated_returns = {
    'mean': list(MC_5_year.simulated_return.mean(axis=1)),
    'median': list(MC_5_year.simulated_return.median(axis=1)),
    'min': list(MC_5_year.simulated_return.min(axis=1)),
    'max': list(MC_5_year.simulated_return.max(axis=1))
}

In [ ]:
commodity_simulated_returns_df = pd.DataFrame(commodity_simulated_returns)

In [ ]:
commodity_simulated_returns_df.head()

In [ ]:
commodity_simulated_returns_df[['mean','median']].plot(title='Simulated cumulative return behavior of gold,silver,and copper over five years')

In [ ]:
initial_investment = 10000

commodities_cumlative_pnl = initial_investment * commodity_simulated_returns_df

commodities_cumlative_pnl.head()

In [ ]:
ci_lower = round(tbl_commodities[8]*10000,2)
ci_upper = round(tbl_commodities[9]*10000,2)

print('there is a 95% chance that an initial investment of $10,000 in these commodities over the next five years will end within the range of')
print('$',{ci_lower} ,'and $',{ci_upper})

In [ ]:
import pandas as pd

# Import the CSV data frame

# Extract the closing prices of the three stocks
gold_prices = gold_df['close']
silver_prices = silver_df['close']
copper_prices = copper_df['close']

# Calculate the daily returns of each stock
gold_returns = np.log(gold_prices) - np.log(gold_prices.shift(1))
silver_returns = np.log(silver_prices) - np.log(silver_prices.shift(1))
copper_returns = np.log(copper_prices) - np.log(copper_prices.shift(1))

# Calculate the weighted daily returns
weighted_returns = (1/3) * (gold_returns + silver_returns + copper_returns)
weighted_returns.dropna()
# Convert the daily returns into cumulative returns
cumulative_returns = (1 + weighted_returns).cumprod()
cumulative_returns.dropna()
# Calculate the portfolio value over time
portfolio_value = 10000 * cumulative_returns

# Retrieve the portfolio value at the last date
final_value = portfolio_value[1259]
print('if you has invested 10,000 evenly into this commodities portfolio on may first 2018 it would be worth $')
print(final_value),print('today')